# Get started tutorial
This notebook ilusrates some of the basic functionality of the package

## Basics

In [ ]:
import pyorb
print(f'pyorb=={pyorb.__version__}')

We first create a standard orbit around the sun in SI units

In [ ]:
orb = pyorb.Orbit(M0 = pyorb.M_sol)

Lets switch to degrees for more human readable units, this can also be given at orbit creation as a keyword parameter

In [ ]:
orb.degrees = True

Currently the orbit has no values

In [ ]:
print(orb)

We can now give it a circular orbit in the plane by calling the `update` method. 

Now when we print the `Orbit` object it will also show the cartesian coordinates as by default conversion between cartesian and kepler elements is automatic. The `Orbit` class also has a number of properties that automatically calculate quantities such as the orbital period from the known elements.

In [ ]:
orb.update(a=1*pyorb.AU, e=0, i=0, omega=0, Omega=0, anom=0)
print(orb)
print(f'Orbital period: {orb.period/(3600.0*24)} days')

We can also look at the position and velocity separatly, for a complete list of properties, see the API documentation

In [ ]:
print(f'Orbit X-position is {orb.x*1e-3} km')
print(f'Orbit velocity vector is [km/s] \n{orb.v*1e-3}')

When accessing the `orb.x` a numpy array of shape `(1,)` is returned. This is due to the fact that the `Orbit` class is naturally vectorized and can support handling `n` orbits simultaniusly. Using this feature greatly reduces computation time when large number of orbits should be converted or modified simulatniusly. We can see the number of orbits currently stored with the `num` property but it also support `len`.

In [ ]:
print(f'Orbit length = {len(orb)}')
print(f'Orbit.num = {orb.num}')

Lastly, we can copy the orbit using the `copy` method.

In [ ]:
orb2 = orb.copy()
print(orb)
print('\n')
print(orb2)

Some useful properties to access are the different kepler anomalies. Here the transfer between them is done by solving the kepler equation. For more information on how the kepler equation is solved, see the `kepler` module in the API documentation. Which anomaly is displayed as `anom` is indicated by the `type` attribute.

In [ ]:
orb2.vz += 3e3
orb2.y += 0.1*pyorb.AU
print(f'orb.type             ="{orb2.type}"')
print(f'orb.anom             ={orb2.anom} deg')
print(f'orb.mean_anomaly     ={orb2.mean_anomaly} deg')
print(f'orb.eccentric_anomaly={orb2.eccentric_anomaly} deg')
print(f'orb.true_anomaly     ={orb2.true_anomaly} deg')

Now, since the updating of cartesian and kepler elements are automatic, as soon as we modify any cartesian elements the new kepler elements will be calculated and stored

In [ ]:
orb.x += 0.1*pyorb.AU
print(orb)

## Updating rules

We can disable this feature using the `auto_update` attribute and the `direct_update` attribute. The `direct_update` decides if a re-calculation of the counterpart elements should be performed as soon as a value is changed while the `auto_update` decides if a recalculation should be performed when an element is accessed. Hence, if `auto_update` does not matter if `direct_update` is set to `True`.

Lets try this out by disabling `direct_update`!

In [ ]:
print('Before:')
print(orb)
orb.direct_update = False
orb.x -= 0.05*pyorb.AU
print('\nAfter:')
print(orb)

As you can see above the cartesian coordinates changed but the kepler did not, they are now in dissagreement. However, internally the `orb` object knows this (using the private `__kep_calculated` array). Now, since `auto_update` is still true, if we access any cartesian variable we will trigger a conversion from kepler to cartesian to harmonize the two coordinate systems.

In [ ]:
print(orb.kepler)

Disabling also this feature will allow us to make transformation between the two systems manually without any tracking of harmony between the systems.

In [ ]:
print('Before:')
print(orb)
orb.auto_update = False
orb.a = 1*pyorb.AU
print('\nAfter:')
print(orb)

We can manually calculate cartesian from kepler using `calculate_cartesian()` or vice-versa using `calculate_kepler()`

In [ ]:
orb.calculate_cartesian()
print(orb)

## Units

We can also create orbits with an arbitrary system of units. Some combinations are implement as standard, otherwise just pass a float that describes the conversion between SI and your unit of choice.

In [ ]:
G_au = pyorb.get_G(length='AU', mass='kg', time='s')
print(f'SI gravitation constant: {pyorb.G} m^3 kg^-1 s^-2')
print(f'Alternative gravitation constant: {G_au} AU^3 kg^-1 s^-2')

We use these units by setting the `G` of the `Orbit` instance. Now we see that both the velocity and positions have changed to AU and AU/year

In [ ]:
orb_customG = pyorb.Orbit(
    M0=pyorb.M_sol, G=G_au, 
    a=1, e=0, i=0, 
    omega=0, Omega=0, anom=0,
)
print(orb_customG)
print(f'Orbital period: {orb_customG.period/(3600.0*24)} days\n')

We can also change this on the fly and apply a common system of units for dynamical astronomy, which is "Astronomical units-Solar masses-years"

In [ ]:
G_ast = pyorb.get_G(length='AU', mass='Msol', time='y')
print(f'Astronomical gravitation constant: {G_ast} AU^3 Msol^-1 y^-2')
orb_customG.G = G_ast

We also need to update the central mass and recalculate the cartesian elements (since mass and gravitational constant updates do not trigger the automatic updating as it only concerns variables, not constants).

Since Kepler elements only have one variable with a physical quantity, the semi-major-axis, and we have not changed the `length` units, it is still "AU", this change only affects the cartesian elements.

In [ ]:
orb_customG.M0 = 1.0
orb_customG.calculate_cartesian()
print(orb2)
print(f'Orbital period: {orb_customG.period} years')

The orbital speed should be approximately $2\pi$ as this is the circumference of a circle with radius 1 AU in units of AU

In [ ]:
print(f'Orbital speed: {orb_customG.velocity} AU/y')